In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, Concatenate, concatenate,Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from tqdm.keras import TqdmCallback
import tensorflow.keras.utils as utils
from datetime import datetime

import cv2
import zipfile
import shutil
import os
import glob, json

from tensorflow.keras.applications import Xception, InceptionV3, MobileNet, ResNet50

import random
import os
from tqdm import tqdm

from ultralytics import YOLO

# Json을 Txt로 

In [ ]:
import json
import os
from tqdm import tqdm

# Dictionary mapping type to class id
type_to_id = { 'd1': 2, 'd2': 2, 'f': 3, 'e': 4}

# Input directories and output directory
input_dirs = ['d1.꽃(개화군)', 'd2.꽃(착과군)', 'e.열매', 'f.만개꽃']
output_dir = 'D:\\097.지능형 스마트팜 통합 데이터(토마토)\\01.데이터\\text'
base_dir = 'D:\\097.지능형 스마트팜 통합 데이터(토마토)\\01.데이터\\1.Training\\라벨링데이터'

# Function to convert polygon to bounding box
def poly_to_bbox(poly_points):
    xs = [point[0] for point in poly_points]
    ys = [point[1] for point in poly_points]
    
    x1 = min(xs)
    y1 = min(ys)
    x2 = max(xs)
    y2 = max(ys)
    
    return [[x1, y1], [x2, y2]]

# Loop over input directories
for input_dir in tqdm(input_dirs, desc="Processing directories"):
    for filename in os.listdir(os.path.join(base_dir, input_dir)):
        if filename.endswith('.json'):
            # Open JSON file
            with open(os.path.join(base_dir, input_dir, filename)) as f:
                data = json.load(f)
                
                # Create directory if not exists
                os.makedirs(os.path.join(output_dir, input_dir), exist_ok=True)

                # Create output .txt file
                with open(os.path.join(output_dir, input_dir, os.path.splitext(filename)[0] + '.txt'), 'w') as outfile:
                    # Loop over shapes in data
                    for shape in data['shapes']:
                        # Get class id from type
                        class_id = type_to_id[data['file_attributes']["type"]]
                        
                        # Convert polygon to bounding box
                        [x1, y1], [x2, y2] = poly_to_bbox(shape['points'])

                        # Calculate YOLO format values
                        x_center = (x1 + x2) / 2.0 / data['imageWidth']
                        y_center = (y1 + y2) / 2.0 / data['imageHeight']
                        width = abs(x2 - x1) / data['imageWidth']
                        height = abs(y2 - y1) / data['imageHeight']

                        # Write to output file
                        outfile.write(f'{class_id} {x_center} {y_center} {width} {height}\n')

# 이름 순서 바꾸기

In [ ]:
root_dir = "D:\\097.지능형 스마트팜 통합 데이터(토마토)\\01.데이터"

def rearrange_filename(filename):
    # Split the filename into parts
    parts = filename.split('_')

    # Rearrange the parts of the filename as per the new order
    rearranged = [parts[0], parts[1], parts[3],parts[5], parts[2], parts[4]] + parts[6:]

    # Join the parts back together and return the result
    return '_'.join(rearranged)

# Pre-calculate total iterations
total_iterations = 0
for index_1 in os.listdir(root_dir):
    for index_2 in os.listdir(os.path.join(root_dir,index_1)):
        for index_3 in os.listdir(os.path.join(root_dir,index_1,index_2)):
            folder_path = os.path.join(root_dir,index_1,index_2,index_3)
            if os.path.isdir(folder_path):
                folder_cat, file_extension = os.path.splitext(folder_path)
                if file_extension != ".zip":
                    if os.path.basename(folder_path) != "TS_Timeseries":
                        total_iterations += len(os.listdir(folder_path))

with tqdm(total=total_iterations) as pbar:
    for index_1 in os.listdir(root_dir):
        for index_2 in os.listdir(os.path.join(root_dir,index_1)):
            for index_3 in os.listdir(os.path.join(root_dir,index_1,index_2)):
                folder_path = os.path.join(root_dir,index_1,index_2,index_3)

                # Ensure it's actually a folder, not a file
                if os.path.isdir(folder_path):
                    folder_cat, file_extension = os.path.splitext(folder_path)
                    if file_extension != ".zip":
                        if os.path.basename(folder_path) != "TS_Timeseries":
                            for file_name in os.listdir(folder_path):
                                old_filepath = os.path.join(folder_path, file_name)
                                # Ensure it's actually a file
                                if os.path.isfile(old_filepath):
                                    # Rearrange the filename
                                    new_filename = rearrange_filename(file_name)
                                    new_filepath = os.path.join(folder_path, new_filename)

                                    # Rename the file
                                    os.rename(old_filepath, new_filepath)
                                    pbar.update()

# 2번 농장 환경정보 전처리

In [ ]:
# Define the file path
tom2 = "D:\\097.지능형 스마트팜 통합 데이터(토마토)\\01.데이터\\1.Training\\원천데이터\\TS_Timeseries\\tom2\\"

#파일 목록보기
print(os.listdir(os.path.join(tom2,"양액기 수집정보")))

# Define a function to detect the encoding of a file
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']
# Pandas로 csv를 호출할 때 csv 인코딩 정보를 사용하도록 해당 csv파일의 인코딩 정보를 추출하는 함수
# 해당 csv는 ANSI, UTF-8(BOM), UTF-8이 혼재. 

# Get the list of file names
tom2_file_list = glob.glob(os.path.join(tom2, "양액기 수집정보", "*"))

# Create an empty dictionary to hold the DataFrames
lp = {}

# Loop over all files in the directory
for file_path in tom2_file_list:
    # Detect the encoding of the file
    encoding = detect_encoding(file_path)

    # Open the file with the detected encoding
    with open(file_path, 'r', encoding=encoding) as f:
        # Read the first line and split it to get column names
        columns = f.readline().strip().split(';')

        # Clean up the column names by replacing double quotes
        columns = [col.replace('"', '') for col in columns]
        columns = [col.strip() for col in columns]

        # Read the rest of the file into a DataFrame
        df = pd.read_csv(f, sep=';', names=columns, header=None)

    # Remove double quotes from data
    df = df.replace({'"': ''}, regex=True)

    # Save the DataFrame in the dictionary
    # Using the filename (without extension) as the key
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    lp[file_name] = df
    
# Time column 변경
for key in lp.keys():
    lp[key]["Time"] = pd.to_datetime(lp[key]["Time"])
    lp[key]["Time"] = lp[key]["Time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    
# MERGE
# Get a sorted list of the keys (optional)
keys = sorted(list(lp.keys()))

# Use the DataFrame from the first CSV file as the starting point
merged = lp[keys[0]]

# Loop over all other DataFrames and merge them into the merged DataFrame
for key in keys[1:]:
    merged = pd.merge(merged, lp[key], on='Time', how='inner')

print(merged.head(3))

# NULL 확인
for col in merged.columns:
    print(f"{col}'s null : {merged[col].isnull().sum()}")

# NULL 전처리를 위해 NULL이 있는 row의 앞뒤 값을 확인
null_indices = lp["MC_temp"][lp["MC_temp"]["Temp_Soil"].isnull()].index

# Expand your indices to include the preceding and following rows
indices = []
for index in null_indices:
    indices.extend([index-1, index, index+1])

# 해당 NULL을 앞 뒤 5개씩 총 10개의 함수의 평균으로 채우기
from IPython.display import display
pd.set_option('display.max_rows', None)  # or any number you prefer
pd.set_option('display.max_columns', None)  # or any number you prefer

surrounding_rows = lp["MC_temp"].loc[indices]
print(surrounding_rows)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

def fill_with_mean(df, column):
    null_indices = df[df[column].isnull()].index
    for idx in null_indices:
        # Check if the previous and next indices are in the dataframe to avoid errors
        if idx - 1 in df.index and idx + 5 in df.index:
            before_value = df.loc[idx - 5, column]
            after_value = df.loc[idx + 5, column]
            
            # Only fill the NaN if both before and after values are not NaN
            if pd.notnull(before_value) and pd.notnull(after_value):
                df.loc[idx, column] = (before_value + after_value) / 5.0
    return df

lp["MC_temp"] = fill_with_mean(lp["MC_temp"], "Temp_Soil")

# txt 제목 바꾸기

In [ ]:
for cat in tqdm(os.listdir(root_dir)):
    for file_name in os.listdir(os.path.join(root_dir, cat)):
        old_path = os.path.join(root_dir, cat, file_name)
        old_name = os.path.basename(old_path)
        parts = old_name.split("_")

        rearranged = [parts[0], parts[1], parts[4], parts[2], parts[5], parts[3]]
        rearranged.extend(parts[6:])
        
        new_name = "_".join(rearranged)
        
        # ensure the extension is preserved
        new_name += os.path.splitext(old_path)[1]

        new_path = os.path.join(root_dir, cat, new_name)

        # rename the file
        os.rename(old_path, new_path)
        
        
# 바꿀 순서 확인
original_name="V001_tom1_39_001_d1_09_20210930_14_03135225_49122255.png"
target_name="V001_tom1_001_09_39_d1_20210930_14_03135225_49122255.txt"

parts=target_name.split("_")

print("Original Name: ", original_name)
part=[parts[0],parts[1],parts[4],parts[2],parts[5],parts[3]]
part.extend(parts[6:])
reassembled_name = "_".join(part)
print("Reassembled Name: ", reassembled_name)
print(type(reassembled_name))